In [16]:
import urllib.request
import json
import pandas as pd

TubeLine = "bakerloo"
DataType = "Pollution"
DataName1 = "PM25"
DataName2 = "HealthIndex"

BASE_URL = "tubesurfers.co.uk"

DataColumns = ['DBid', 'Timestamp', 'Line', 'StationFrom', 'StationTo', 'IndexBetweenStations', 'DBid2', 'TimestampID', 'DataName', 'DataValue', 'DateType', 'Units', 'SensorID']

URL = "http://" +BASE_URL+ "/line_data/" + TubeLine + "/" + DataType + "/" + DataName1
data1 = pd.DataFrame(json.loads(urllib.request.urlopen(URL).read()), columns=DataColumns)

URL = "http://" +BASE_URL+ "/line_data/" + TubeLine + "/" + DataType + "/" + DataName2
data2 = pd.DataFrame(json.loads(urllib.request.urlopen(URL).read()), columns=DataColumns)

def TidyDataFrame(df: pd.DataFrame):
    df['DataValue'] = df['DataValue'].map(lambda val: float(val))

TidyDataFrame(data1)
TidyDataFrame(data2)

orderedTubeStops = ["Willesden Green", "Kensal Green", "Queen's Park", "Kilburn Park","Maida Vale", "Warwick Avenue", "Paddington", "Edgware Road", "Marylebone", "Baker Street", "Regent's Park", "Oxford Circus", "Charing Cross", "Embankment", "Waterloo", "Lambeth North", "Elephant & Castle"]

def OrderDataFrame(df: pd.DataFrame):
    sortedDF = pd.DataFrame([], columns=DataColumns)
    for idx, tubeStop in enumerate(orderedTubeStops):
        stationDF = df.where(df['StationFrom'] == tubeStop)
        stationDF.dropna()
        stationDF = stationDF.where(stationDF['StationTo'] == tubeStop)

        for node in stationDF.iterrows():
            pd.concat(sortedDF, pd.DataFrame(node, columns=DataColumns))


        if (len(orderedTubeStops) < idx):
            between_stationDF = df.where(df['StationFrom'] == tubeStop)
            stationDF.dropna()
            between_stationDF = between_stationDF.where(between_stationDF['StationTo'] == orderedTubeStops[idx+1])

            for node in between_stationDF.iterrows():
                pd.concat(sortedDF, pd.DataFrame(node, columns=DataColumns))

    df = sortedDF

OrderDataFrame(data1)
OrderDataFrame(data2)

ValueError: Shape of passed values is (2, 1), indices imply (2, 13)